In [ ]:
import cv2 as cv
%load_ext autoreload
%autoreload 2
import os
import sys; sys.path.append('../')
import matplotlib.pyplot as plt

import pandas as pd
import math
import seaborn as sns;
import numpy as np
import tensorflow as tf
from keras import backend as K
from tensorflow import keras
from scipy.spatial.distance import cdist
from tensorflow.keras.optimizers import Adam
from src.utils.keypoints_filter import mean_distance_filter
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras import layers, regularizers
from tqdm import tqdm
import pickle
from tqdm import tqdm
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 120
import sys; sys.path.append('../')
color = {
        1:"brown",
        2:"blue",
        3:"green",
        4:"red"
}
import matplotlib.image as mpimg
data_path = "../data/Ki67/SHIDC-B-Ki-67/Train"
img_name= "p1_0299_6"

In [ ]:
def grey_world(nimg):
    nimg = nimg.transpose(2, 0, 1).astype(np.uint32)
    mu_g = np.average(nimg[1])
    nimg[0] = np.minimum(nimg[0]*(mu_g/np.average(nimg[0])),255)
    nimg[2] = np.minimum(nimg[2]*(mu_g/np.average(nimg[2])),255)
    return  nimg.transpose(1, 2, 0).astype(np.uint8)

In [ ]:
def create_keypoints_dict(img_name,data_path,radius=None,plot=False):
    img_path = os.path.join(str(data_path),str(img_name)+".jpg")
    json_path = os.path.join(str(data_path),img_name+".json")
    df = pd.read_json(json_path)
    img = grey_world(mpimg.imread(img_path))
    auto = False
    if plot:
        plt.imshow(img)

        for idx , row in df.iterrows():
                x = row[0]; y = row[1]; label = row[2]
                plt.plot(x, y, marker='*', color=color[label])
        plt.show()
    sift = sift = cv.SIFT_create(
            nOctaveLayers=40,
            contrastThreshold=0.02,
            edgeThreshold=30,
            sigma=1.3)
    keypoints, desc = sift.detectAndCompute(img,None)
    
    
    points = list(zip(df.x,df.y))
    if len(points)<=0:
        return None
    
    if radius is None:
        d=cdist(points,points)
        d=d.flatten()
        d.sort()
        radius = d[d>0][0]*0.5
        auto=True
        
    
    keys = [k.pt for k in keypoints]
    distances = cdist(points,keys)
    
    x=np.argwhere(distances <=radius)
    df_features = pd.DataFrame()
    if plot:
        plt.imshow(img)
        
    for pair in x:
        point_idx = pair[0]
        keypoint_idx = pair[1]
        label = df.label_id[point_idx]
        keypoint_dict = {
            'angle':keypoints[keypoint_idx].angle,
            'response':keypoints[keypoint_idx].response,
            'size':keypoints[keypoint_idx].size,
            'octave':keypoints[keypoint_idx].octave,
            'desc':desc[keypoint_idx],
            'label': label
        }
        df_features = df_features.append(keypoint_dict,ignore_index=True)
        
        if plot:
            plt.plot(keypoints[keypoint_idx].pt[0],keypoints[keypoint_idx].pt[1],marker='o',color=color[label])
            plt.plot(points[point_idx][0],points[point_idx][1],marker='^',color=color[label])
    if auto:
        distant=np.where(np.all(distances>radius*2,axis=0))[0]
    else:
        distant=np.where(np.all(distances>radius*4,axis=0))[0]
    #print(len(distant))
    for distant_key_idx in distant:
        if keypoints[distant_key_idx].response > 0.002:
            if plot:
                plt.plot(keypoints[distant_key_idx].pt[0],keypoints[distant_key_idx].pt[1],marker='o',color=color[4])
            keypoint_dict = {
                'angle':keypoints[distant_key_idx].angle,
                'response':keypoints[distant_key_idx].response,
                'size':keypoints[distant_key_idx].size,
                'octave':keypoints[distant_key_idx].octave,
                'desc':desc[distant_key_idx],
                'label': 4
            }
            df_features = df_features.append(keypoint_dict,ignore_index=True)
    return df_features

In [ ]:
img_path = os.path.join(str(data_path),str(img_name)+".jpg")
json_path = os.path.join(str(data_path),img_name+".json")
df = pd.read_json(json_path)
#img = grey_world()
plt.imshow(mpimg.imread(img_path))

In [ ]:
d = create_keypoints_dict(img_name,data_path,plot=True,radius=8)

In [ ]:
d.set_index('label').mean(level=0)
#df.sort_values(by=['size'],ascending=False).head(100)

In [ ]:
d.response.min()

In [ ]:
d.response.max()

In [ ]:
path=r"../data/Ki67/SHIDC-B-Ki-67/"
path_type = ["Test","Train"]   

In [ ]:
for data_type in path_type:
    df_all = pd.DataFrame()
    root_path = os.path.join(str(path),data_type)
    for root, dirs, files in os.walk(root_path, topdown=False):
        for file in tqdm(files):
            if file.split('.')[1]=='json':
                json_path = os.path.join(root,file)
                size = os.path.getsize(json_path)
                if size > 2:
                    try:
                        img_name = file.split('.')[0]
                        dic = create_keypoints_dict(img_name,data_path=root_path,radius=8)
                        if not dic.empty:
                            df_all = df_all.append(dic,ignore_index=True)
                        pass
                    except Exception as e:
                        print("error with file: ",img_name,' exc: ',e)
                    
                else:
                    #print('empty json', json_path)
                    pass
    with open(r'../keypoints_desc_0'+data_type+'.pickle', 'wb') as handle:
        pickle.dump(df_all, handle, protocol=pickle.HIGHEST_PROTOCOL)                
    df_all.to_csv(r'../keypoints_descs_0'+data_type+'.csv',index=False)

In [ ]:
df_all.to_csv(r'../keypoints_descs_0Train'+'.csv',index=False)

In [ ]:
corr = df_all.corr()
sns.heatmap(corr, 
        xticklabels=corr.columns,
        yticklabels=corr.columns)

# Dataloading

In [172]:
with open(r'../keypoints_desc_0Train.pickle', 'rb') as handle:
    train_set = pickle.load(handle)
with open(r'../keypoints_desc_0Test.pickle', 'rb') as handle:
    test_set = pickle.load(handle)  
train_set.loc[train_set.label !=4, ['label']]=0
train_set.loc[train_set.label ==4, ['label']]=1

test_set.loc[test_set.label !=4, ['label']]=0
test_set.loc[test_set.label ==4, ['label']]=1

test_set=test_set.sample(frac = 1)

train_set=train_set.sample(frac=1)

In [173]:
y_train = train_set.label.values
y_test = test_set.label.values
train_set=train_set.drop(columns=['label'],axis=1)
test_set=test_set.drop(columns=['label'],axis=1)

In [174]:
from sklearn.preprocessing import normalize
def process_set(df):
    all_items=[]
    for index, row in tqdm(df.iterrows()):
        items = []
        #print(row[1])
        #items.append(row[0])#angle
        items.extend(row[1]) #desc
      #  items.append(row[2]) #octave
       # items.append(row[3]) #response
      #  items.append(row[4]) #size
        all_items.append(items)
    all_items=normalize(all_items,axis=1)
    return all_items

In [175]:
tf.random.set_seed(0)
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(physical_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [176]:
class_weights

array([2.12779375, 0.65358211])

In [177]:

x_list = process_set(train_set)
x = tf.convert_to_tensor(x_list)

x_test_list = process_set(test_set)
x_test = tf.convert_to_tensor(x_test_list)


787522it [00:53, 14857.69it/s]
337259it [00:21, 15676.84it/s]


In [181]:
x_list[2]

array([0.00390285, 0.03512565, 0.09757124, 0.22246242, 0.06634844,
       0.05659132, 0.05854274, 0.02536852, 0.2283167 , 0.22636527,
       0.11123121, 0.15806541, 0.02731995, 0.0234171 , 0.09171696,
       0.21465672, 0.2283167 , 0.04878562, 0.        , 0.        ,
       0.        , 0.00390285, 0.00585427, 0.08586269, 0.01365997,
       0.00195142, 0.        , 0.        , 0.00195142, 0.00390285,
       0.00390285, 0.01951425, 0.03317422, 0.02146567, 0.00975712,
       0.02536852, 0.04293134, 0.12879403, 0.2283167 , 0.09757124,
       0.14830828, 0.06049417, 0.02146567, 0.10342551, 0.08391126,
       0.04683419, 0.2283167 , 0.2283167 , 0.2283167 , 0.05854274,
       0.0078057 , 0.03317422, 0.02146567, 0.00585427, 0.01170855,
       0.05463989, 0.01951425, 0.00195142, 0.        , 0.        ,
       0.00195142, 0.        , 0.        , 0.0078057 , 0.05073704,
       0.04878562, 0.13659973, 0.04683419, 0.0156114 , 0.02536852,
       0.04097992, 0.02146567, 0.08976554, 0.12293976, 0.10342

# Classic - baseline models

In [ ]:
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(x_test_list,y_test)

In [ ]:
y_pred = neigh.predict(x_test_list)
y_true=y_test
print(classification_report(y_true, y_pred))

In [ ]:
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(x_list, y_train)
y_pred = clf.predict(x_test_list)
y_true=y_test
print(classification_report(y_true, y_pred))

# Random check

In [ ]:
import random
y_true=y_test
#zero check what is model only print out 1 class 
random_output = [random.randint(0, 1) for i in range(len(y_true))]
ones = np.array([1 for item in y_true])
zeros = [0 for item in y_true]
print(classification_report(y_true, ones))
print(classification_report(y_true, zeros))
print(classification_report(y_true, random_output))

# Deep models

In [186]:
import time
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils import class_weight
from sklearn.metrics import f1_score
from tensorflow.keras import regularizers

In [271]:
BATCH_SIZE=4096
EPOCH = 100
from sklearn.metrics import classification_report
def check_model(model,y_true,x_test):
    y_pred=model.predict_classes(x_test)
    print(classification_report(y_true, y_pred))

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
class_weight_dict = dict(enumerate(class_weights))

c:\users\wilk\appdata\local\programs\python\python39\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0. 1.], y=[0. 1. 1. ... 0. 1. 1.] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [187]:
def create_model(UNITS,DROPS,REGS):
    name="model_seq"
    model = tf.keras.models.Sequential()
    units_it = 0
    drops_it = 0
    while units_it < len(UNITS) and drops_it < len(DROPS):
        if units_it < len(UNITS):
            if units_it ==0:
                model.add(Dense(units=UNITS[units_it], input_shape=(x.shape[1],), activation="relu", name="input_layer"))
                name+="_"+str(UNITS[units_it])
            else:
                if REGS[units_it]==True:
                    model.add(Dense(UNITS[units_it],kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01), activation="relu"))
                    name+="_"+str(UNITS[units_it])+"reg"
                else:
                    model.add(Dense(UNITS[units_it], activation="relu"))
                    name+="_"+str(UNITS[units_it])
                
            units_it +=1
        if drops_it < len(DROPS):
            model.add(Dropout(DROPS[drops_it]))
            model.add(BatchNormalization())
            name+="_"+str(DROPS[drops_it])
            drops_it += 1
            
    model._name = name
    model.add(Dense(units=2, activation="softmax"))
    model.build()
    return model

In [272]:
def train(UNITS,DROPS,REGS,lr=0.001,return_model=False,trial=None):
    results = pd.DataFrame()
    model = create_model(UNITS,DROPS,REGS)
    if trial != None:
        MODEL_NAME = str(trial.number)+"_"+model.name
    else:
         MODEL_NAME = model.name
    save_path = f"./best_models/"+MODEL_NAME
    model_path = save_path+f"/model"
    tb_callback = tf.keras.callbacks.TensorBoard(save_path, update_freq='epoch')
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=save_path+f"/model",
                                                     save_weights_only=True,
                                                     verbose=0,
                                                     save_best_only=True)
    

    model.compile(optimizer=Adam(learning_rate=lr),loss='sparse_categorical_crossentropy',metrics=['acc'],run_eagerly=False,)
    #print(EPOCH,BATCH_SIZE)
    history_pre = model.fit(
    x,y_train,batch_size=BATCH_SIZE,
    validation_data=(x_test,y_test),
    epochs=EPOCH,class_weight=class_weight_dict,
    callbacks=[tb_callback,cp_callback],verbose=0
    )
    model.load_weights(model_path)
    #check_model(model,y_train,x)
    #check_model(model,y_test,x_test)
    test_results = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE,return_dict=True)
    train_results = model.evaluate(x, y_train, batch_size=BATCH_SIZE,return_dict=True)
    y_pred = tf.argmax(model.predict(x_test),axis=1)
    fs = f1_score(y_test,y_pred)
    #print(train_results)
    raport = classification_report(y_test, y_pred,output_dict=True)
    mean_fs= (raport['1.0']['f1-score']+raport['0.0']['f1-score'])/2
    res = {
        'model_name':[MODEL_NAME],
        'val_loss':test_results['loss'],
        'val_acc':test_results['acc'],
        'train_loss':train_results['loss'],
        'train_acc':train_results['acc'],
        'fscore':[fs],
        'mean_fs':[mean_fs]
    }
    model.save(save_path+f"/model")
    results=results.append(res,ignore_index=True)
    if return_model:
        return results, model
    return results

In [273]:
import tensorflow as tf

import optuna
MAX_UNITS = 1024
MAX_DROP = 0.5

# 1. Define an objective function to be maximized.
def objective(trial):
    BATCH_SIZE= trial.suggest_int('BATCH_SIZE', 256, 8096)    
    # 2. Suggest values of the hyperparameters using a trial object.
    n_dense_layers = trial.suggest_int('n_layers_dense', 1, 8)
    n_drop_layers = trial.suggest_int('n_layers_dropout', 1, n_dense_layers)
    UNITS = []
    DROPS = []
    REGS = []
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    for i in range(n_dense_layers):
        UNITS.append(int(trial.suggest_loguniform('n_units_Dense_l_{}'.format(i), 4, MAX_UNITS)))
        REGS.append(trial.suggest_categorical('n_regularization_{}'.format(i), [True,False]))
            
    for i in range(n_drop_layers):
        DROPS.append(float(trial.suggest_loguniform('n_units_Drop_l_{}'.format(i), 0.1, MAX_DROP)))
    res = train(UNITS,DROPS,REGS,lr,False,trial)
    print(res)
    return res['mean_fs'][0]

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2021-04-23 23:14:22,056] A new study created in memory with name: no-name-3f4ce134-0280-43e2-87d2-014c98e55399


193/193 [==============================] - 1s 4ms/step - loss: 0.4084 - acc: 0.8212
INFO:tensorflow:Assets written to: ./best_models/0_model_seq_39_0.23716372306907105/model\assets


[I 2021-04-23 23:17:01,286] Trial 0 finished with value: 0.7563986212207403 and parameters: {'BATCH_SIZE': 1534, 'n_layers_dense': 7, 'n_layers_dropout': 1, 'lr': 0.005251357185118188, 'n_units_Dense_l_0': 39.418440255888456, 'n_regularization_0': True, 'n_units_Dense_l_1': 69.91576889054491, 'n_regularization_1': True, 'n_units_Dense_l_2': 6.966963988076822, 'n_regularization_2': False, 'n_units_Dense_l_3': 427.0120971707229, 'n_regularization_3': True, 'n_units_Dense_l_4': 652.83416768919, 'n_regularization_4': False, 'n_units_Dense_l_5': 259.6182969446069, 'n_regularization_5': True, 'n_units_Dense_l_6': 366.726225968632, 'n_regularization_6': False, 'n_units_Drop_l_0': 0.23716372306907105}. Best is trial 0 with value: 0.7563986212207403.


                 fscore               mean_fs  \
0  [0.8772036233176748]  [0.7563986212207403]   

                             model_name  train_acc  train_loss   val_acc  \
0  [0_model_seq_39_0.23716372306907105]   0.821234    0.408378  0.816307   

   val_loss  
0  0.412464  
193/193 [==============================] - 1s 6ms/step - loss: 0.4454 - acc: 0.8135
INFO:tensorflow:Assets written to: ./best_models/1_model_seq_282_0.47159760926767524_113_0.27863587144970037_32_0.2642952272636736_9reg_0.23540725746259403_5reg_0.13061552285242256_6_0.23029977568681706/model\assets


[I 2021-04-23 23:23:06,507] Trial 1 finished with value: 0.7478589022745681 and parameters: {'BATCH_SIZE': 4831, 'n_layers_dense': 8, 'n_layers_dropout': 6, 'lr': 2.3145548645576707e-05, 'n_units_Dense_l_0': 282.65764081342775, 'n_regularization_0': False, 'n_units_Dense_l_1': 113.78484035766238, 'n_regularization_1': False, 'n_units_Dense_l_2': 32.7183158489799, 'n_regularization_2': False, 'n_units_Dense_l_3': 9.656008367755447, 'n_regularization_3': True, 'n_units_Dense_l_4': 5.289389788466928, 'n_regularization_4': True, 'n_units_Dense_l_5': 6.799371188114847, 'n_regularization_5': False, 'n_units_Dense_l_6': 75.31740016040999, 'n_regularization_6': True, 'n_units_Dense_l_7': 87.76923706369647, 'n_regularization_7': False, 'n_units_Drop_l_0': 0.47159760926767524, 'n_units_Drop_l_1': 0.27863587144970037, 'n_units_Drop_l_2': 0.2642952272636736, 'n_units_Drop_l_3': 0.23540725746259403, 'n_units_Drop_l_4': 0.13061552285242256, 'n_units_Drop_l_5': 0.23029977568681706}. Best is trial 0 w

...

                fscore               mean_fs  \
0  [0.906202039925269]  [0.7754384776366767]   

                                          model_name  train_acc  train_loss  \
0  [97_model_seq_295_0.37960850559893855_22_0.276...   0.860423     0.35739   

    val_acc  val_loss  
0  0.851583  0.368158  
193/193 [==============================] - 1s 7ms/step - loss: 0.3764 - acc: 0.8630
INFO:tensorflow:Assets written to: ./best_models/98_model_seq_237_0.10686069068259929_55_0.15635146566406852_575_0.3265403267107762_29reg_0.27126622047520077_55reg_0.336715066953836_9reg_0.4920836295699712_161_0.16308741617776507/model\assets


[I 2021-04-24 10:32:51,680] Trial 98 finished with value: 0.7796631058316486 and parameters: {'BATCH_SIZE': 1376, 'n_layers_dense': 8, 'n_layers_dropout': 7, 'lr': 0.005382929030819486, 'n_units_Dense_l_0': 237.52271191275614, 'n_regularization_0': True, 'n_units_Dense_l_1': 55.990119195140984, 'n_regularization_1': False, 'n_units_Dense_l_2': 575.5433006780964, 'n_regularization_2': False, 'n_units_Dense_l_3': 29.687612301956705, 'n_regularization_3': True, 'n_units_Dense_l_4': 55.38575151874117, 'n_regularization_4': True, 'n_units_Dense_l_5': 9.090565646239465, 'n_regularization_5': True, 'n_units_Dense_l_6': 161.01911494959316, 'n_regularization_6': False, 'n_units_Dense_l_7': 455.776093756244, 'n_regularization_7': False, 'n_units_Drop_l_0': 0.10686069068259929, 'n_units_Drop_l_1': 0.15635146566406852, 'n_units_Drop_l_2': 0.3265403267107762, 'n_units_Drop_l_3': 0.27126622047520077, 'n_units_Drop_l_4': 0.336715066953836, 'n_units_Drop_l_5': 0.4920836295699712, 'n_units_Drop_l_6': 0

                 fscore               mean_fs  \
0  [0.9074955997643199]  [0.7796631058316486]   

                                          model_name  train_acc  train_loss  \
0  [98_model_seq_237_0.10686069068259929_55_0.156...   0.863021    0.376387   

    val_acc  val_loss  
0  0.853827  0.389126  
193/193 [==============================] - 1s 6ms/step - loss: 0.3458 - acc: 0.8632
INFO:tensorflow:Assets written to: ./best_models/99_model_seq_141_0.10426904618073357_28_0.15444939164885968_800_0.3887425584629123_42reg_0.276704237651837/model\assets


[I 2021-04-24 10:39:08,556] Trial 99 finished with value: 0.7777263649351385 and parameters: {'BATCH_SIZE': 795, 'n_layers_dense': 6, 'n_layers_dropout': 4, 'lr': 0.005367814659810099, 'n_units_Dense_l_0': 141.57105341371778, 'n_regularization_0': True, 'n_units_Dense_l_1': 28.336226572099584, 'n_regularization_1': False, 'n_units_Dense_l_2': 800.9248070049412, 'n_regularization_2': False, 'n_units_Dense_l_3': 42.064708742409444, 'n_regularization_3': True, 'n_units_Dense_l_4': 53.42980675750326, 'n_regularization_4': True, 'n_units_Dense_l_5': 9.245926506960842, 'n_regularization_5': True, 'n_units_Drop_l_0': 0.10426904618073357, 'n_units_Drop_l_1': 0.15444939164885968, 'n_units_Drop_l_2': 0.3887425584629123, 'n_units_Drop_l_3': 0.276704237651837}. Best is trial 69 with value: 0.7806392230795393.


                 fscore               mean_fs  \
0  [0.9063635100161144]  [0.7777263649351385]   

                                          model_name  train_acc  train_loss  \
0  [99_model_seq_141_0.10426904618073357_28_0.154...   0.863215    0.345755   

    val_acc  val_loss  
0  0.852173  0.366171  


In [274]:
study.__dict__

{'study_name': 'no-name-3f4ce134-0280-43e2-87d2-014c98e55399',
 '_study_id': 0,
 '_storage': <optuna.storages._in_memory.InMemoryStorage at 0x2ef7f7ecb80>,
 'sampler': <optuna.samplers._tpe.sampler.TPESampler at 0x2ef7f7ec1c0>,
 'pruner': <optuna.pruners._median.MedianPruner at 0x2ef098bbf40>,
 '_optimize_lock': <unlocked _thread.lock object at 0x000002EF098BB960>,
 '_stop_flag': False}

True

In [195]:
model = keras.models.load_model(best_model_path)

OSError: SavedModel file does not exist at: ./best_model_ever\{saved_model.pbtxt|saved_model.pb}

In [197]:
model.load_weights(best_model_path)

OSError: Unable to open file (unable to open file: name = './best_model_ever', errno = 13, error message = 'Permission denied', flags = 0, o_flags = 0)

In [198]:
dic = {'BATCH_SIZE': 306, 'n_layers_dense': 7, 'n_layers_dropout': 7, 'lr': 0.010735916535015695, 'n_units_Dense_l_0': 204.81884666203757, 'n_regularization_0': True, 'n_units_Dense_l_1': 652.7110125045335, 'n_regularization_1': True, 'n_units_Dense_l_2': 792.3879971600081, 'n_regularization_2': False, 'n_units_Dense_l_3': 107.89558566379694, 'n_regularization_3': True, 'n_units_Dense_l_4': 41.59020705944122, 'n_regularization_4': False, 'n_units_Dense_l_5': 117.29893362528279, 'n_regularization_5': True, 'n_units_Dense_l_6': 318.7395482960147, 'n_regularization_6': False, 'n_units_Drop_l_0': 0.14526986897124874, 'n_units_Drop_l_1': 0.33541798789822896, 'n_units_Drop_l_2': 0.12025767363520917, 'n_units_Drop_l_3': 0.15023335263456342, 'n_units_Drop_l_4': 0.21906920051269188, 'n_units_Drop_l_5': 0.12191408797721591, 'n_units_Drop_l_6': 0.13014311378250998}

In [199]:
dic

{'BATCH_SIZE': 306,
 'n_layers_dense': 7,
 'n_layers_dropout': 7,
 'lr': 0.010735916535015695,
 'n_units_Dense_l_0': 204.81884666203757,
 'n_regularization_0': True,
 'n_units_Dense_l_1': 652.7110125045335,
 'n_regularization_1': True,
 'n_units_Dense_l_2': 792.3879971600081,
 'n_regularization_2': False,
 'n_units_Dense_l_3': 107.89558566379694,
 'n_regularization_3': True,
 'n_units_Dense_l_4': 41.59020705944122,
 'n_regularization_4': False,
 'n_units_Dense_l_5': 117.29893362528279,
 'n_regularization_5': True,
 'n_units_Dense_l_6': 318.7395482960147,
 'n_regularization_6': False,
 'n_units_Drop_l_0': 0.14526986897124874,
 'n_units_Drop_l_1': 0.33541798789822896,
 'n_units_Drop_l_2': 0.12025767363520917,
 'n_units_Drop_l_3': 0.15023335263456342,
 'n_units_Drop_l_4': 0.21906920051269188,
 'n_units_Drop_l_5': 0.12191408797721591,
 'n_units_Drop_l_6': 0.13014311378250998}

In [202]:
UNITS = [int(204.81884666203757),int(652.7110125045335),int(792.3879971600081),int(107.89558566379694),int(41.59020705944122),int(117.29893362528279),int(318.7395482960147)]

In [203]:
DROPS = [float(0.14526986897124874),float(0.33541798789822896),float(0.12025767363520917),float()]

[204, 652, 792, 107, 41, 117, 318]

In [207]:
UNITS = []
DROPS = []
REGS = []
for key in dic.keys():
    if key.startswith('n_units_Dense'):
        UNITS.append(int(dic[key]))
    if key.startswith('n_units_Drop'):
        DROPS.append(float(dic[key]))
    if key.startswith('n_regularization'):
        REGS.append(dic[key])
        

In [208]:
UNITS

[204, 652, 792, 107, 41, 117, 318]

In [209]:
DROPS

[0.14526986897124874,
 0.33541798789822896,
 0.12025767363520917,
 0.15023335263456342,
 0.21906920051269188,
 0.12191408797721591,
 0.13014311378250998]

In [210]:
REGS

[True, True, False, True, False, True, False]

In [211]:
model = create_model(UNITS,DROPS,REGS)

In [252]:
BATCH_SIZE = 306
EPOCH = 1
res, model =train(UNITS,DROPS,REGS,0.010735916535015695,True)

1 306
2574/2574 [==============================] - 12s 5ms/step - loss: 0.6664 - acc: 0.7819
INFO:tensorflow:Assets written to: ./best_models/model_seq_204_0.14526986897124874_652reg_0.33541798789822896_792_0.12025767363520917_107reg_0.15023335263456342_41_0.21906920051269188_117reg_0.12191408797721591_318_0.13014311378250998/model\assets


In [227]:
model

In [229]:
check_model(model,y_train,x)
check_model(model,y_test,x_test)
test_results = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE,return_dict=True)
train_results = model.evaluate(x, y_train, batch_size=BATCH_SIZE,return_dict=True)
y_pred = tf.argmax(model.predict(x_test),axis=1)
fs = f1_score(y_test,y_pred)

C:\Users\Wilk\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

         0.0       0.87      0.47      0.61    185056
         1.0       0.86      0.98      0.91    602466

    accuracy                           0.86    787522
   macro avg       0.86      0.72      0.76    787522
weighted avg       0.86      0.86      0.84    787522



C:\Users\Wilk\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

         0.0       0.87      0.47      0.61     82426
         1.0       0.85      0.98      0.91    254833

    accuracy                           0.85    337259
   macro avg       0.86      0.72      0.76    337259
weighted avg       0.86      0.85      0.84    337259

2574/2574 [==============================] - 13s 5ms/step - loss: 0.5293 - acc: 0.8590


In [230]:
fs

0.9093596814284618

In [231]:
test_results

{'loss': 0.5347673892974854, 'acc': 0.8527363538742065}

In [241]:
res = classification_report(y_test, y_pred,output_dict=True)
(res['1.0']['f1-score']+res['0.0']['f1-score'])/2

0.7584832095477803

In [246]:
model.save("my_model")

INFO:tensorflow:Assets written to: my_model\assets
